In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('forums/*.json')
len(files)

90

In [3]:
forums = []
for f in files:
    with open(f) as fopen:
        forums.extend(json.load(fopen))
        
forums = sorted(list(set(forums)))
len(forums)

1670976

In [5]:
def get_url(url, topic):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if topic in a_ and '&page=' in a_]
    [a_.split('&extra=page%3D1&page=')[-1] for a_ in a]
    max_page = []
    for a_ in a:
        try:
            max_page.append(int(a_.split('&extra=page%3D1&page=')[-1]))
        except:
            pass

    try:
        max_page = max(max_page)
    except:
        max_page = -1
    td = soup.find_all('td', {'class': 't_f'})
    td = [str(t) for t in td]
    return url, td, max_page

In [6]:
!mkdir threads

mkdir: cannot create directory ‘threads’: File exists


In [ ]:
max_worker = 20

for i in tqdm(range(0, len(forums), max_worker)):
    
    filename = os.path.join('threads', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
        
    topics = [('https://c.cari.com.my/' + topic, topic) for topic in forums[i: i + max_worker]]
    
    data = []    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url[0], url[1]): url for url in topics}

        for future in as_completed(futures):
            data.append(future.result())
                
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)
        
    try:
        del data
        del futures
    except:
        pass

  6%|█▉                               | 4782/83549 [1:49:20<32:19:38,  1.48s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 12%|███▉                            | 10386/83549 [3:59:12<27:25:05,  1.35s/it]encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE4 0xBB
encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE4 0xBB
encoding error : input conversion failed due to input error, bytes 0x8F 0xAF 0xE4 0xBB
 15%|████▋                           | 12247/83549 [4:42:51<25:24:43,  1.28s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x90 0xE7 0xAD
encoding error : input conversion failed due to input error, bytes 0x9D 0x90 0xE7 0xAD


In [ ]:
len(data)

In [ ]:
data[-4]